In [1]:
! pip install pandas-datareader

In [2]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import datetime
import FinanceDataReader as fdr
import matplotlib.pyplot as plt 
import seaborn as sns    

In [3]:
start = datetime.datetime(2020, 8, 8)
end = datetime.datetime.today()

finance = {"kakaobank":"323410", "TYM":"002900", "shinwon":"009270", "skbio":"302440", "indf":"014900", "iyeon":"102460", "gwangjushinseagea":"037710", "sajo":"008040", "deadong":"008110", "ygplus":"037270", "fund_10":"152380", "kbstar" : "136340", "gold":"132030", "wti":"261220","k_inverse":"114800"}
for i in finance.keys():
    globals()['{}'.format(i)] = fdr.DataReader(finance[i],start,end)

for i in finance.keys():
    print(globals()['{}'.format(i)])

             Open   High    Low  Close    Volume  Change
Date                                                    
2021-08-06  53700  69800  51000  69800  59340894     NaN
            Open  High   Low  Close     Volume    Change
Date                                                    
2020-08-10  1175  1250  1170   1195    7606665  0.025751
2020-08-11  1200  1215  1170   1175    3009715 -0.016736
2020-08-12  1180  1185  1135   1165    2279758 -0.008511
2020-08-13  1180  1185  1160   1170    2032775  0.004292
2020-08-14  1195  1235  1180   1205    7480122  0.029915
...          ...   ...   ...    ...        ...       ...
2021-08-02  2115  2125  2055   2055    2668349 -0.023753
2021-08-03  2055  2085  2035   2055    1697807  0.000000
2021-08-04  2075  2215  2060   2145   11658517  0.043796
2021-08-05  2175  2210  2130   2130    7311196 -0.006993
2021-08-06  2170  2735  2150   2610  149449091  0.225352

[247 rows x 6 columns]
            Open  High   Low  Close     Volume    Change
Date   

In [4]:
df_list = [globals()['{}'.format(i)]['Close'] for i in finance.keys()]

KeyError: 'Close'

In [ ]:
stock = pd.concat(df_list, axis=1)
stock.columns = finance.keys()
stock_corr = stock.corr()

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(data = stock_corr, annot=True, 
fmt = '.2f', linewidths=.5, cmap='Blues')

In [ ]:
cor_list = list(stock_corr['k_inverse'])

In [ ]:
def weight(x):
    wp = []
    for i in x:
        wp.append(100*(1-i)/(sum([1-j for j in x])))

    col = stock_corr.columns
    df = pd.DataFrame(wp, col, columns=["weight"])
    
    return df

In [ ]:
weight(cor_list)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_coef(df):
    s_cols = list(df.columns)
    coef = []
    for i in range(len(s_cols)):
        X_data = df[df.columns.difference([s_cols[i]])]
        y_data = df[s_cols[i]]
        X_data.fillna(X_data.mean(), inplace=True)
        y_data.fillna(y_data.mean(), inplace=True)

        model = LinearRegression()
        model.fit(X_data, y_data)
        c = model.coef_
        c = np.insert(c, i, 0)
        coef.append(c)

    coef_df = pd.DataFrame(coef, columns = s_cols, index = s_cols)
    return coef_df

In [ ]:
stock_coef = get_coef(stock)

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(data = stock_coef, annot=True, 
fmt = '.2f', linewidths=.5, cmap='Blues')

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller

In [ ]:
kospi

In [ ]:
adfuller_test = adfuller(kospi['Close'], autolag= "AIC")
print("ADF test statistic: {}".format(adfuller_test[0]))
print("p-value: {}".format(adfuller_test[1]))

In [ ]:
adfuller_test = adfuller(kospi['Close'].diff().dropna(), autolag= "AIC")
print("ADF test statistic: {}".format(adfuller_test[0]))
print("p-value: {}".format(adfuller_test[1]))

In [ ]:
adfuller_test = adfuller(kospi['Volume'].diff().dropna(), autolag= "AIC")
print("ADF test statistic: {}".format(adfuller_test[0]))
print("p-value: {}".format(adfuller_test[1]))

In [ ]:
model = VAR(kospi.diff(1).dropna())

In [ ]:
results = model.fit(1)

In [ ]:
results.summary()

In [ ]:
def arima(df):
    from statsmodels.tsa.arima_model import ARIMA
    import statsmodels.api as sm
    model = ARIMA(df['Close'], order=(2,1,2))
    model_fit = model.fit(trend='nc')
    fore = model_fit.forecast(steps=10)
    forecast = pd.DataFrame(fore[0], columns=['df'])
    return forecast

In [ ]:
arima(sp500)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, TimeDistributed, RepeatVector, Dropout, Bidirectional, BatchNormalization

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler
import tempfile
import time
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
kospi.corr()['Close']

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

plt.rcParams['figure.figsize'] = [12, 8]
seasonal_decompose(kospi['Close'], model='additive', period = 30).plot()
plt.show()

In [ ]:
kospi.corr()

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
kospi_df = kospi.iloc[:,:4]

In [ ]:
split_date = int(kospi_df.shape[0]*0.8)
train = kospi_df[:split_date]
test = kospi_df[split_date:]

#TempAvgF scale
scaler = MinMaxScaler()
train_scale = pd.DataFrame(scaler.fit_transform(train), columns=kospi_df.columns) 
test_scale = pd.DataFrame(scaler.transform(test), columns=kospi_df.columns)
# We should use transform in TEST data. If we use fit_transform, model learns test data too.

train_feature = train_scale[train_scale.columns.difference(['Close'])]
train_label = train_scale[['Close']]
test_feature = test_scale[test_scale.columns.difference(['Close'])]
test_label = test_scale[['Close']]

# Split data each Timesteps
def make_dataset(data, label, window_size, predict_size):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size-predict_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size:i+window_size+predict_size]))
    return np.array(feature_list), np.array(label_list)

timeseteps = 30
predict_size = 5

train_feature, train_label = make_dataset(train_feature, train_label, timeseteps, predict_size)
test_feature, test_label = make_dataset(test_feature, test_label, timeseteps, predict_size)

train_label = train_label.reshape(train_label.shape[0],train_label.shape[1])
test_label = test_label.reshape(test_label.shape[0],test_label.shape[1])

In [ ]:
from attention import Attention

def setup_model(n, lr):
    model = Sequential()
    model.add(GRU(n,activation='relu',return_sequences=True,input_shape=(train_feature.shape[1],train_feature.shape[2])))
    model.add(GRU(n,activation='relu'))
    Attention()
    model.add(Dense(predict_size, activation='linear'))
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=lr))

    return model

model = setup_model(n=64, lr=0.005)
batch_size = 10

model.fit(train_feature, train_label,batch_size=batch_size,epochs=100,validation_split=0.2, verbose=0)

print(model.evaluate(test_feature, test_label))

predict = model.predict(test_feature)
predict = predict.reshape(predict.shape[0],predict.shape[1])
print("rmse: {}".format(np.sqrt(mean_squared_error(test_label, predict))))
print('r2: {}'.format(r2_score(test_label, predict)))

In [ ]:
pd.DataFrame(predict).iloc[:,4].plot()

In [ ]:
kospi['indicator'] = kospi['Volume']*kospi['Change']
kospi_df = kospi
split_date = int(kospi_df.shape[0]*0.8)
train = kospi_df[:split_date]
test = kospi_df[split_date:]

#TempAvgF scale
scaler = MinMaxScaler()
train_scale = pd.DataFrame(scaler.fit_transform(train), columns=kospi_df.columns) 
test_scale = pd.DataFrame(scaler.transform(test), columns=kospi_df.columns)
# We should use transform in TEST data. If we use fit_transform, model learns test data too.

train_feature = train_scale[['Volume', 'Change']]
train_label = train_scale[['indicator']]
test_feature = test_scale[['Volume', 'Change']]
test_label = test_scale[['indicator']]


timeseteps = 30
predict_size = 5

train_feature, train_label = make_dataset(train_feature, train_label, timeseteps, predict_size)
test_feature, test_label = make_dataset(test_feature, test_label, timeseteps, predict_size)

train_label = train_label.reshape(train_label.shape[0],train_label.shape[1])
test_label = test_label.reshape(test_label.shape[0],test_label.shape[1])

In [ ]:
def setup_model(n, lr):
    model = Sequential()
    model.add(GRU(n,activation='relu',return_sequences=True,input_shape=(train_feature.shape[1],train_feature.shape[2])))
    model.add(GRU(n,activation='relu'))
    Attention()
    model.add(Dense(predict_size, activation='linear'))
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=lr))

    return model

model = setup_model(n=32, lr=0.005)
batch_size = 10

model.fit(train_feature, train_label,batch_size=batch_size,epochs=100,validation_split=0.2, verbose=0)

print(model.evaluate(test_feature, test_label))

predict = model.predict(test_feature)
predict = predict.reshape(predict.shape[0],predict.shape[1])
print("rmse: {}".format(np.sqrt(mean_squared_error(test_label, predict))))
print('r2: {}'.format(r2_score(test_label, predict)))